날짜 : 23/03/10

코드 복사해서 혼자 공부하려고 주석 달아놓은 자료

In [ ]:
import numpy as np

BOARD_ROWS = 3
BOARD_COLS = 4
WIN_STATE = (0, 3) # 교재에 써놓은 것처럼 위에서 아래로 바라본 array 인덱스 나타낸거임. 어려운거 아님
LOSE_STATE = (1, 3)
START = (2, 0)
DETERMINISTIC = False

class environment:
    def __init__(self, state=START):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.board[1, 1] = -1
        self.state = state
        self.isEnd = False
        self.determine = DETERMINISTIC

    def giveReward(self): # win state에 도달하면 reward 1을 주고, trap에 빠지면 -1dmf wna
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0

    def isEndFunc(self):
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.isEnd = True

    def _chooseActionProb(self, action): # Random Environment를 구현하기 위한 확률분포 구축 (dynamics) 
    # 이부분은 Noise가 구현된 것임
        if action == "up":
            return np.random.choice(["up", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "down":
            return np.random.choice(["down", "left", "right"], p=[1.0, 0.0, 0.0])
        if action == "left":
            return np.random.choice(["left", "up", "down"], p=[1.0, 0.0, 0.0])
        if action == "right":
            return np.random.choice(["right", "up", "down"], p=[1.0, 0.0, 0.0])

    def nextPosition(self, action): # Agent에게서 action을 받으면, 일단은 스토캐스틱이기 때문에 (non d) 를 통해 정해진 액션 방향으로 무조건 흘러가게끔 함. (takeAction)
        """
        action: up, down, left, right
        -------------
        0 | 1 | 2| 3|
        1 |
        2 |
        return next position on board
        """
        if self.determine:
            if action == "up":
                nextState = (self.state[0] - 1, self.state[1])
            elif action == "down":
                nextState = (self.state[0] + 1, self.state[1])
            elif action == "left":
                nextState = (self.state[0], self.state[1] - 1)
            else:
                nextState = (self.state[0], self.state[1] + 1)
            self.determine = False 
            # 이부분이 tricky하다고 말씀하심. t시점의 스토캐스틱이 t+1의 스토캐스틱이랑 연결되면 또다른 스토캐스틱이 생기기 때문에 그렇게 하지 않기 위해서 자름.
            # 새로운 스토캐스틱을 생성하지 않게끔 해주는 것임.**조건부에 조건부를 곱하면 베이즈 정리에 의해 새로운 분포가 형성되기 때문(이부분이 핵심인듯)
            # non d에서는 내가 취한 action이 chooseActionPrbo에서 고른 액션으로 바뀌게 됨.(dynamics를 통해 바뀐 액션으로 바뀜)
            # 그럼 걔가 그 모드를 (non d를) d로 바꿔주면, next state는 deterministic으로 다시 action을 시행한다. 
        else:
            # non-deterministic
            action = self._chooseActionProb(action)
            self.determine = True
            nextState = self.nextPosition(action)

        # if next state is legal (벽이 아니면)
        # 벽을 마주쳤을때 벽으로 가지 않게끔.
        # state12개로 한거임. 벽을 마주치면 0을 움직임. 벽을 하나의 state로 구현해놓은 것임.
        if (nextState[0] >= 0) and (nextState[0] <= 2):
            if (nextState[1] >= 0) and (nextState[1] <= 3):
                if nextState != (1, 1):
                    return nextState
        return self.state

    def showBoard(self): # 현제 스테이트가 어떤 스테이트인지 plot해주는거라 이해하는데는 크게 상관 없는 코드
        self.board[self.state] = 1
        for i in range(0, BOARD_ROWS):
            print('-----------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = '*'
                if self.board[i, j] == -1:
                    token = 'z'
                if self.board[i, j] == 0:
                    token = '0'
                out += token + ' | '
            print(out)
        print('-----------------')

In [ ]:
env = environment()

In [ ]:
env.showBoard()

-----------------
| 0 | 0 | 0 | 0 | 
-----------------
| 0 | z | 0 | 0 | 
-----------------
| * | 0 | 0 | 0 | 
-----------------


Env 코드는 Agent가 action을 취했을때 interaction이 가능한 환경을 구축해 놓은것임(playground)

**Env** -> State, Reward. -> **Agent** -> action

Agent는 random 하게 돌아다닌다고 가정(어떤 분포로?)



In [ ]:
class randomAgent:
    def __init__(self):
        self.states = []  # record position and action taken at the position
        self.actions = ["up", "down", "left", "right"]
        self.env = environment()
        self.isEnd = self.env.isEnd
        self.result_stat = []

    def randomAction(self):# 액션을 하나 choose
        action = np.random.choice(self.actions)
        return action

    def takeAction(self, action): # Agent가 취한 액션을 Env의 NextPosition에다가 전달해줘서 움직이게 만들어줌.
        position = self.env.nextPosition(action)
        return environment(state=position)

    def reset(self):
        self.states = []
        self.env = environment()
        self.isEnd = self.env.isEnd

    def play(self, rounds): # 게임이 끝날때까지 플레이 해라.
        i=0
        self.result_stat = []
        while i < rounds:
            if self.env.isEnd:
                reward = self.env.giveReward()
                if reward >= 1:
                    self.result_stat.append(reward)
                self.reset()
                i += 1
            else:
                action = self.randomAction()
                self.states.append([(self.env.state), action])
                self.env = self.takeAction(action)
                self.env.isEndFunc()
                self.isEnd = self.env.isEnd
        success_rate = np.sum(self.result_stat) / rounds
        print("Success rate:{} %".format(success_rate * 100))

In [ ]:
agent = randomAgent()
agent.play(rounds=1000)
# >>> Success rate: 35.2% ± 2.9%

Success rate:29.7 %


더 스마트한 AGENT를 만드려면 `def randomACtion(self):`를 바꿔줘야함.

random action(following Uniform Distribution)을 따르는 것이 아니라, 적절한 random action을 따르는 걸 Assignment로 해보시오. 


**Agent Action의 적절한 전략은 무엇일까?** ⭐

해당 코드에서 Action Choice는 totally random이다.

Reward, State record에 대한 데이터가 전혀 사용되고 있지 않다. 이것을 활용하면 Success rate을 더 높일수 있지 않을까?

리워드의 누적합 최대화하는 목적 = 에이전트가 달성하고자 하는 목적과 동치